## Feature Selection

- Create scikit learn basic pipeline
- Standardize the train data (Done in 8)
- Options :
    - Regularization (glmnet) (Lasso, Ridge, Elastic Net)
    - PCA , MCA
    - Trees - Feature Importance (Random Forest - with filter, Decision Tree)
    - VIF testing
    - Forward Selection
    - Backward Selection    
    - Recursive Feature Elimination
    - Feature Importance (Random Forest, Decision Tree)
    - Correlation Matrix
    
